In [ ]:
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
from pathlib import Path
from espnet2.text.phoneme_tokenizer import PhonemeTokenizer
from espnet2.text.token_id_converter import TokenIDConverter
from espnet2.bin.tts_inference import Text2Speech

%load_ext autoreload
%autoreload 2

PWD = %pwd
PWD = Path(PWD)
LJSPEECH_DIR = (PWD / '../egs2/ljspeech/tts1/').resolve()
device = 'cuda'

In [6]:
os.chdir(LJSPEECH_DIR)
pretrained_dir = LJSPEECH_DIR / "exp/tts_train_jets_raw_phn_tacotron_g2p_en_no_space"
pretrained_model_file = pretrained_dir / "train.total_count.ave_5best.pth"
pretrained_tts = Text2Speech.from_pretrained(
    train_config=pretrained_dir / "config_prosody.yaml",
    model_file=pretrained_model_file,
    device=device
)
pretrained_model = pretrained_tts.model
os.chdir(PWD)

/home/perry/miniconda3/envs/espnet/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


It seems weight norm is not applied in the pretrained model but the current model uses it. To keep the compatibility, we remove the norm from the current model. This may cause unexpected behavior due to the parameter mismatch in finetuning. To avoid this issue, please change the following parameters in config to false:
 - discriminator_params.follow_official_norm
 - discriminator_params.scale_discriminator_params.use_weight_norm
 - discriminator_params.scale_discriminator_params.use_spectral_norm

See also:
 - https://github.com/espnet/espnet/pull/5240
 - https://github.com/espnet/espnet/pull/5249


In [7]:
arpa_tokenizer = PhonemeTokenizer(g2p_type='g2p_en_no_space')
id_converter = TokenIDConverter(pretrained_tts.train_args.token_list)

In [3]:
from en_to_de import phonemize
phoneset = set()
with open('/home/perry/PycharmProjects/datasets/CSS10/german/transcript_phones_espeak.txt', 'w') as phones_f:
    with open('/home/perry/PycharmProjects/datasets/CSS10/german/transcript.txt') as f:
        for line in f:
            filename, _, text2, _ = line.split('|')
            # phones = german_tokenizer.text2tokens(text2)
            phones = phonemize(text2)
            phones_f.write(f'{filename}|{" ".join(phones)}\n')
            phoneset |= set(phones)

In [4]:
' '.join(sorted(phoneset))

'! (de) (en) , . 1 ? a aɪ aʊ b d dʒ eː f h i iː j k l m n oː p pf s t ts tʃ uː v x y yː z | ç øː ŋ œ ɐ ɑː ɔ ɔø ə əʊ ɛ ɛː ɜ ɡ ɪ ʁ ʃ ʊ ʒ ˈa ˈaɪ ˈaʊ ˈeː ˈiː ˈoː ˈuː ˈy ˈyː ˈøː ˈœ ˈɑ ˈɑː ˈɔ ˈɔø ˈɔː ˈɛ ˈɛː ˈɪ ˈʊ ˌa ˌaɪ ˌaʊ ˌeː ˌiː ˌoː ˌuː ˌy ˌyː ˌøː ˌœ ˌɑ ˌɑː ˌɔ ˌɔø ˌə ˌɛ ˌɛː ˌɪ ˌʊ'

In [58]:
from en_to_de import PHONES_MAP, PUNCS
phoneset = set(PHONES_MAP.keys()) | set(PUNCS)
with open('/home/perry/PycharmProjects/datasets/CSS10/german/transcript_phones_espeak.txt') as f:
    for line in f:
        filename, trans = line.split('|')
        for phone in trans.strip().split():
            if phone not in phoneset:
                phoneset.add(phone)

In [59]:
with open('/home/perry/PycharmProjects/datasets/CSS10/german/transcript_phones_espeak.txt') as f:
    for line in f:
        filename, trans = line.split('|')
        line_phones = set(trans.strip().split(' '))
        if line_phones & phoneset:
            print(filename, list(line_phones))
            phoneset -= line_phones
            if not phoneset:
                break

achtgesichterambiwasse/achtgesichterambiwasse_0000.wav ['p', 'ˈaɪ', 'n', 'aɪ', 'l', 'ɪ', 'z', 'ɑː', 'ˈœ', 't', 'a', 'ɐ', 'ˌa', 'ˈɪ', 'ˈiː', 'ɡ', 'ˈa', 's', 'k', 'ə', 'ˈɛː', 'ts', 'h', 'ç', 'ɜ', 'ʃ', 'ˈɑ', 'eː', 'ˌʊ', 'm', 'j', 'ʊ', 'd', 'ˈuː', ',']
achtgesichterambiwasse/achtgesichterambiwasse_0001.wav ['ˈaɪ', 'n', 'aɪ', 'l', 'ɪ', 'uː', 'z', 'ɑː', 'ˈɛ', '.', 'ˈœ', 't', 'ɐ', 'ˌaɪ', 'ˈiː', 'ʁ', 'ɡ', 'v', 'iː', 's', 'k', 'ə', 'ts', 'ˈɛː', 'ç', 'h', 'ɜ', 'ˌɛ', 'ˈɑː', 'ʃ', 'eː', 'm', 'b', 'ˌeː', 'ɛ', 'f', 'ʊ', 'ŋ', 'd', ',']
achtgesichterambiwasse/achtgesichterambiwasse_0002.wav ['p', 'ˈaɪ', 'n', 'ˈy', 'l', 'ɪ', 'z', 'ˈɛ', '.', 't', 'ɐ', 'x', 'aʊ', 'ˌaɪ', 'ˈɪ', 'ʁ', 'ɡ', 'ˈa', 'ˈeː', 'v', 'iː', 's', 'k', 'ə', 'ts', 'ç', 'ˈɑː', 'ˈɔ', 'ʃ', 'ˈyː', 'm', 'b', 'ˌiː', 'f', 'ɛ', 'ʊ', 'ŋ', 'ˈʊ', 'd', ',']
achtgesichterambiwasse/achtgesichterambiwasse_0003.wav ['n', 'aɪ', 'ɪ', 'l', 'z', 'uː', '.', 't', 'ɐ', 'aʊ', 'oː', 'ˈoː', 'ˌaɪ', 'ˈɪ', 'ˈiː', 'ˌɪ', 'ɡ', 'ʁ', 'ˈa', 'v', 'iː', 's', 'k', 'ɔ', 'ə', 't

In [8]:
line_dict = {}
filenames = set('''
achtgesichterambiwasse/achtgesichterambiwasse_0000.wav 
achtgesichterambiwasse/achtgesichterambiwasse_0001.wav 
achtgesichterambiwasse/achtgesichterambiwasse_0002.wav 
achtgesichterambiwasse/achtgesichterambiwasse_0003.wav 
achtgesichterambiwasse/achtgesichterambiwasse_0006.wav 
achtgesichterambiwasse/achtgesichterambiwasse_0008.wav 
achtgesichterambiwasse/achtgesichterambiwasse_0009.wav 
achtgesichterambiwasse/achtgesichterambiwasse_0011.wav 
achtgesichterambiwasse/achtgesichterambiwasse_0014.wav 
achtgesichterambiwasse/achtgesichterambiwasse_0020.wav 
achtgesichterambiwasse/achtgesichterambiwasse_0022.wav 
achtgesichterambiwasse/achtgesichterambiwasse_0024.wav 
achtgesichterambiwasse/achtgesichterambiwasse_0026.wav 
achtgesichterambiwasse/achtgesichterambiwasse_0034.wav 
achtgesichterambiwasse/achtgesichterambiwasse_0052.wav 
achtgesichterambiwasse/achtgesichterambiwasse_0063.wav 
achtgesichterambiwasse/achtgesichterambiwasse_0065.wav 
achtgesichterambiwasse/achtgesichterambiwasse_0086.wav 
achtgesichterambiwasse/achtgesichterambiwasse_0099.wav 
achtgesichterambiwasse/achtgesichterambiwasse_0153.wav 
achtgesichterambiwasse/achtgesichterambiwasse_0192.wav 
achtgesichterambiwasse/achtgesichterambiwasse_0258.wav 
achtgesichterambiwasse/achtgesichterambiwasse_0363.wav 
achtgesichterambiwasse/achtgesichterambiwasse_1029.wav 
achtgesichterambiwasse/achtgesichterambiwasse_2189.wav
meisterfloh/meisterfloh_0452.wav
meisterfloh/meisterfloh_1928.wav
'''.strip().split())

with open('/home/perry/PycharmProjects/datasets/CSS10/german/transcript.txt') as f:
    for line in f:
        filename, _, trans, _ = line.split('|')
        if filename in filenames:
            line_dict[filename] = trans
            filenames.remove(filename)
            if not filenames:
                break

In [62]:
line_dict

{'achtgesichterambiwasse/achtgesichterambiwasse_0000.wav': 'Hanake hatte allen Körperschmuck, den ein japanisches Mädchen sitzend, trippelnd und liegend zeigen muß,',
 'achtgesichterambiwasse/achtgesichterambiwasse_0001.wav': 'um zu den göttlichen Schönheiten der Vergänglichkeit gezählt zu werden. Ihr Hals war biegsam wie eine Reiherfeder,',
 'achtgesichterambiwasse/achtgesichterambiwasse_0002.wav': 'ihre Arme kurz wie die Flügel eines noch nicht flüggen Sperlings. Saß sie auf der Matte und bereitete ihren Tee,',
 'achtgesichterambiwasse/achtgesichterambiwasse_0003.wav': 'so arbeitete sie vorsichtig wie unter einer Glasglocke. Ging sie abends mit ihrer Dienerin auf den hohen Holzschuhen zum Theater,',
 'achtgesichterambiwasse/achtgesichterambiwasse_0006.wav': 'Lag sie in der Nacht hinter den geschlossenen Papierwänden ihres Hauses mit frisiertem Kopf auf der Schlummerrolle',
 'achtgesichterambiwasse/achtgesichterambiwasse_0008.wav': 'als wäre es aus Jadestein geschnitten und erschien u

In [5]:
schwa_lines = '''achtgesichterambiwasse/achtgesichterambiwasse_1334.wav
meisterfloh/meisterfloh_2263.wav
achtgesichterambiwasse/achtgesichterambiwasse_0599.wav
achtgesichterambiwasse/achtgesichterambiwasse_2176.wav'''.split()

In [6]:
r_lines = '''achtgesichterambiwasse/achtgesichterambiwasse_1009.wav
achtgesichterambiwasse/achtgesichterambiwasse_2242.wav
achtgesichterambiwasse/achtgesichterambiwasse_1166.wav'''.split()

In [29]:
a_lines='''achtgesichterambiwasse/achtgesichterambiwasse_1415.wav
achtgesichterambiwasse/achtgesichterambiwasse_0813.wav
serapionsbruederauswahl/serapionsbruederauswahl_0681.wav
serapionsbruederauswahl/serapionsbruederauswahl_0592.wav
serapionsbruederauswahl/serapionsbruederauswahl_1023.wav'''.split()

In [31]:
fname_phones = {}
with open('/home/perry/PycharmProjects/datasets/CSS10/german/transcript_phones_espeak.txt') as f:
    for line in f:
        fname, phones = line.strip().split('|')
        fname_phones[fname] = phones
a_phones = {fname: fname_phones[fname] for fname in a_lines}
r_phones = {fname: fname_phones[fname] for fname in r_lines}

In [8]:
from en_to_de import GermanArpaSpeech, phonemize
gas = GermanArpaSpeech(token_id_converter=id_converter, tts_inference_fn=pretrained_model.tts.generator.inference)

In [26]:
from en_to_de import custom_ipa_subs

In [28]:
phones = tuple(''' ˌiː z ə n | k n ˈɑː b ə n | n ˈuː n | d ɑː ʁ ˈaʊ s | v ˌɪ ɐ t | n ˈɪ ç t s | , f ˈuː ɐ | p ˈeː ʁ ɛ ɡ ɐ ˌiː n ʊ s | , d ɛ ɐ | l ˈɛ ŋ s t | v ˈʊ s t ə | , v ˌoː | d iː | ˈa l t ə | h ɪ n ˈaʊ s v ɔ l t ˌə | , h ˈɛ f t ɪ ç | ˈaʊ f |'''.strip().split())
' '.join(custom_ipa_subs(phones))

'ˌiː z ə n | k n ˈɑː b ə n | n ˈuː n | d ɑː ʁ ˈaʊ s | v ˌɪ ɐ t | n ˈɪ ç t s | , f ˈuː ɐ | p ˈeː ʁ ɛ ɡ ʁ ˌiː n ʊ s | , d ɛ ɐ | l ˈɛ ŋ s t | v ˈʊ s t ə | , v ˌoː | d iː | ˈa l t ə | h ɪ n ˈaʊ s v ɔ l t ˌə | , h ˈɛ f t ɪ ç | ˈaʊ f |'

In [31]:
from en_to_de import CONSONANTS_MAP, VOWELS_MAP
print(''.join(sorted(set(''.join(set(CONSONANTS_MAP))))), ''.join(sorted(set(''.join(set(VOWELS_MAP))))))

bdfhjklmnpstvxzçŋɡʁʃʒ aeiouyøœɐɑɔəɛɜɪʊˈˌ


In [33]:
# phones_map = {'a': 'AH2',
#     'ˌa': ('AH1', '1', '1'),
#     'ˈa': ('AH1', '1', '2'),}
for fname, phones in a_phones.items():
    german = phones.split()
    inputs = gas.gen_audio(
        german,
        save_dir=PWD / 'outputs/tts_train_jets_raw_phn_tacotron_g2p_en_no_space/CSS10/german',
        verbose=False,
        # phones_map=phones_map,
        # verbose=True,
        custom_filename=fname,
        device=device,
    )

In [23]:
gas.gen_inputs('Ata-Mono', verbose=True)

   -1      0    1           2            3           4            5           6
 arpas    AA1   T          AA0           M          OW1           N          OW0
d_factor  0.7  1.0  1.0499999999999998  1.0  1.0499999999999998  1.0  1.0499999999999998
e_factor  0.0  0.0         0.0          0.0         0.0          0.0         0.0


(array(['AA1', 'T', 'AA0', 'M', 'OW1', 'N', 'OW0'], dtype='<U3'),
 tensor([[0.7000, 1.0000, 1.0500, 1.0000, 1.0500, 1.0000, 1.0500]],
        device='cuda:0', dtype=torch.float64),
 tensor([[0., 0., 0., 0., 0., 0., 0.]], device='cuda:0', dtype=torch.float64))

In [27]:
german = 'Ata-Mono fragte rasch: Kennst du die Rindensprache der roten Kryptomerienbäume? Natürlich , sagte die Frau ebenso rasch.'
arpas, d_factor, e_factor = gas.gen_inputs(german)
d_factor[0, 1] *= 1.5
inputs = gas.gen_audio(
        german,
        inputs=(arpas, d_factor, e_factor),
        save_dir=PWD / 'outputs/tts_train_jets_raw_phn_tacotron_g2p_en_no_space/CSS10/german',
        verbose=False,
        # verbose=True,
        custom_filename=fname,
        device=device,
    )

In [18]:
' '.join(gas.convert_phones_arpa(phonemize('Ata-Mono fragte rasch: Kennst du die Rindensprache der roten Kryptomerienbäume? Natürlich , sagte die Frau ebenso rasch.'))[0].tolist())

'AA1 T AA0 M OW1 N OW0 F HH R AA1 K T AH0 , HH R AH1 SH , K EH1 N S T D W UW0 D IY0 HH R IH1 N D AH0 N SH P HH R AA2 HH K HH AH0 , D EH2 AH0 , HH R OW1 T AH0 N , K HH R Y UH1 P T OW0 M EY2 HH R IH0 EH2 N B OY0 M AH0 , ? N AA0 T Y UH1 AH0 L IH0 H SH S , Z AA1 G T AH0 , D IY0 F HH R AA1 W , EY1 B AH0 N Z OW2 HH R AH1 SH .'

In [24]:
# Try on the first line of the CSS10 German dataset.
filenames = sorted(line_dict.keys())
filename = filenames[26]
german = line_dict[filename]

phones = phonemize(german)

print(filename, german, ' '.join(phones), ' '.join(gas.convert_phones_arpa(phones)[0]), sep='\n')

inputs = gas.gen_audio(
    german,
    save_dir=PWD / 'outputs/tts_train_jets_raw_phn_tacotron_g2p_en_no_space/german',
    verbose=False,
    # verbose=True,
    device=device,
)

meisterfloh/meisterfloh_1928.wav
Diesen Knaben nun – Daraus wird nichts, fuhr Peregrinus, der längst wußte, wo die Alte hinauswollte, heftig auf
d ˌiː z ə n | k n ˈɑː b ə n | n ˈuː n | d ɑː ʁ ˈaʊ s | v ˌɪ ɐ t | n ˈɪ ç t s | , f ˈuː ɐ | p ˈeː ʁ ɛ ɡ ɐ ˌiː n ʊ s | , d ɛ ɐ | l ˈɛ ŋ s t | v ˈʊ s t ə | , v ˌoː | d iː | ˈa l t ə | h ɪ n ˈaʊ s v ɔ l t ˌə | , h ˈɛ f t ɪ ç | ˈaʊ f |
D IY2 Z AH0 N , K N AA1 B AH0 N , N W UW1 N D AA0 HH W R AA1 W S V IH2 AH0 T N IH1 H SH S T S , F W UW1 AH0 , P EY1 HH W R EH2 G AH1 IY2 N W UH0 S , D EH2 AH0 , L EH1 NG S T V W UH1 S T AH0 , , V OW2 D IY0 AH1 L T AH0 , HH IH0 N AA1 W S V AO0 L T AH0 , , HH EH1 F T IH0 H SH S AA1 W F


In [10]:
from tqdm import tqdm
# Now run on entire dataset
save_dir = PWD / 'outputs/tts_train_jets_raw_phn_tacotron_g2p_en_no_space/CSS10/german'
transcript_file = (PWD / '../../datasets/CSS10/german/transcript_phones_espeak.txt').resolve()
with open(transcript_file) as f:
    for line in tqdm(f.read().splitlines()):
        custom_filename, phones = line.strip().split('|', maxsplit=1)
        if (save_dir / custom_filename).exists():
            continue
        german = phones.split()
        
        inputs = gas.gen_audio(
            german,
            save_dir=save_dir,
            verbose=False,
            custom_filename=custom_filename,
            # verbose=True,
            device=device,
        )
        # arpas, d_factor, e_factor = inputs


100%|██████████| 7427/7427 [26:02<00:00,  4.75it/s]


In [34]:
p = ' '.join(phonemize('die Arme umeinander legten und, Wange an Wange, ihr Blut aneinander pochen ließen.'))